# Tagesaufgaben Python und PostgreSQL

## Aufgabe 1 - Postgres Bedienung


In [1]:
import psycopg
import pandas as pd
from configparser import ConfigParser
from getpass import getpass
import sqlalchemy
from sqlalchemy import text


**a)** Erstelle eine Postgres-Datenbank "programmiersprachen" und verbinde dich zu ihr.
(Sollte vorher bereits eine bestehen, lösche diese erst!)

Hinweis: Sollte sich die Datenbank nicht erstellen lassen, stelle autocommit auf True.

In [48]:
connection = psycopg.connect(
	host='localhost',
	port='5432',
	user='postgres',
	password='42',
	dbname='programmiersprachen',
	autocommit=True
)

In [6]:
create_db = ('CREATE DATABASE programmiersprachen;')

In [7]:
cursor = connection.cursor()
cursor.execute(create_db)

<psycopg.Cursor [COMMAND_OK] [IDLE] (host=localhost database=postgres) at 0x15c10cf8350>

In [15]:
cursor.close()
connection.close()

**b)** Erstelle in der neuen Datenbank eine Tabelle namens `programmiersprachen` und füge die folgenden Inhalte ein:


In [8]:
data = [
	(1, "Ada Lovelace Machinealgorithm", 1843),
	(2, "Assembler", 1949),
	(3, "COBOL", 1959),
	(4, "SQL", 1972),
	(5, "C++", 1985),
	(6, "Python", 1991),
	(7, "Swift", 2014)
]

In [18]:
cursor = connection.cursor()
cursor.execute(
	'''CREATE TABLE IF NOT EXISTS programmiersprachen(
	id int,
	prog_language VARCHAR(50) UNIQUE NOT NULL,
	release_year int);'''
)

<psycopg.Cursor [COMMAND_OK] [IDLE] (host=localhost user=postgres database=programmiersprachen) at 0x15c10cf9250>

In [21]:
cursor.executemany('''
                  INSERT INTO programmiersprachen(id, prog_language, release_year)
                  VALUES (%s, %s, %s)''', data)

**c)** Lasse dir die Inhalte der Tabelle als pandas DataFrame anzeigen.


In [23]:
prog_lang_df = pd.read_sql('SELECT * FROM programmiersprachen', connection)
prog_lang_df

C:\Users\Admin\AppData\Local\Temp\ipykernel_15600\2762793855.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  prog_lang_df = pd.read_sql('SELECT * FROM programmiersprachen', connection)


,id,prog_language,release_year
0,1,Ada Lovelace Machinealgorithm,1843
1,2,Assembler,1949
2,3,COBOL,1959
3,4,SQL,1972
4,5,C++,1985
5,6,Python,1991
6,7,Swift,2014


## Aufgabe 2 - Funktionen


**a)** Erstelle eine SQL Tabelle `student` mit den Spalten id, Vorname, Nachname und Alter


In [79]:
cursor = connection.cursor()
cursor.execute(
	'''CREATE TABLE IF NOT EXISTS student(
	id SERIAL PRIMARY KEY,
	vorname VARCHAR(20),
	nachname VARCHAR(20),
	alter INT NOT NULL
	)'''
)

<psycopg.Cursor [COMMAND_OK] [IDLE] (host=localhost user=postgres database=programmiersprachen) at 0x15c112f2810>

**b)** Schreibe eine Funktion, mit der du Studenten in deine neue SQL Tabelle eintragen kannst. Teste sie mit einem Aufruf.


In [80]:
def add_students(vorname, nachname, alter):
	insert_q = """
		INSERT INTO student (vorname, nachname, alter)
		VALUES (%s, %s, %s);
	"""
	cursor.execute(insert_q, (vorname, nachname, alter))
	connection.commit()

In [81]:
add_students("Mitch", "Maddison", 39)

In [82]:
cursor.execute('SELECT * FROM student;')

<psycopg.Cursor [TUPLES_OK] [IDLE] (host=localhost user=postgres database=programmiersprachen) at 0x15c112f2810>

In [83]:
student_table = cursor.fetchall()
student_table

[(1, 'Mitch', 'Maddison', 39)]

**c)** Schreibe eine Funktion zum Löschen eines Studenten anhand einer ID. Teste die Funktion mit einem Testaufruf. Optional: Die Infos der gelöschten ID sollen dabei auch angezeigt werden.

In [84]:
def kill_students(connection, id):
	with connection.cursor() as cc:
		cc.execute("DELETE FROM student WHERE id = %s", (id,))
		connection.commit()
		print(f"student with id {id} deleted from table!👍🏻")

In [85]:
kill_students(connection, 1)

student with id 1 deleted from table!👍🏻


**d)** Schließe die Datenbank-, Cursor und/oder Engine-Verbindung.

In [86]:
cursor.close()
connection.close()